In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

print("PyTorch version:", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

PyTorch version: 1.10.0
Using cpu device


In [2]:
epochs = 12

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Flatten(),
            nn.Linear(96*96, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = BasicCNN().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test loss: {test_loss}, Test Acc: {correct*100:.2f}% \n")
    

/Users/aalhaimi/env/pttest/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:189.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
##### SMALLER BATCH SIZE ######

batch_size = 32

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
%%time

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)


Epoch 1
-------------------------------
Test loss: 0.05483208737362506, Test Acc: 97.93% 

Epoch 2
-------------------------------
Test loss: 0.04334579336858597, Test Acc: 98.50% 

Epoch 3
-------------------------------
Test loss: 0.044429291182587476, Test Acc: 98.66% 

Epoch 4
-------------------------------
Test loss: 0.04648422347321039, Test Acc: 98.76% 

Epoch 5
-------------------------------
Test loss: 0.054129266863941414, Test Acc: 98.56% 

Epoch 6
-------------------------------
Test loss: 0.043046030674961586, Test Acc: 98.86% 

Epoch 7
-------------------------------
Test loss: 0.040471177259869054, Test Acc: 98.97% 

Epoch 8
-------------------------------
Test loss: 0.053516180126911096, Test Acc: 98.77% 

Epoch 9
-------------------------------
Test loss: 0.04947723142399773, Test Acc: 98.83% 

Epoch 10
-------------------------------
Test loss: 0.04336354392557107, Test Acc: 99.05% 

Epoch 11
-------------------------------
Test loss: 0.04393908693543405, Test Acc: 9

In [5]:
##### BIGGER BATCH SIZE ######
#### simulating larger images with higher memory reqs

batch_size = 4096

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [6]:
%%time

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)


Epoch 1
-------------------------------
Test loss: 0.42984333634376526, Test Acc: 86.31% 

Epoch 2
-------------------------------
Test loss: 0.28651319940884906, Test Acc: 91.38% 

Epoch 3
-------------------------------
Test loss: 0.21845370531082153, Test Acc: 93.49% 

Epoch 4
-------------------------------
Test loss: 0.1624086151520411, Test Acc: 95.25% 

Epoch 5
-------------------------------
Test loss: 0.12017902731895447, Test Acc: 96.38% 

Epoch 6
-------------------------------
Test loss: 0.0888558030128479, Test Acc: 97.35% 

Epoch 7
-------------------------------
Test loss: 0.07028715188304584, Test Acc: 97.76% 

Epoch 8
-------------------------------
Test loss: 0.059385862201452255, Test Acc: 98.10% 

Epoch 9
-------------------------------
Test loss: 0.05204996590813001, Test Acc: 98.28% 

Epoch 10
-------------------------------
Test loss: 0.0474063865840435, Test Acc: 98.43% 

Epoch 11
-------------------------------
Test loss: 0.04421733692288399, Test Acc: 98.49% 
